In [65]:
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer

In [66]:
df_train = pd.read_csv('train.csv')

In [67]:
## Data Preprocessing, removing null values
df_train= df_train.dropna()

In [68]:
import re
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

df_train['DESCRIPTION'] = df_train['DESCRIPTION'].apply(lambda x: remove_tags(x))

In [69]:
df_train.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,"[Simple and elegant, great for displaying indo...",HINS Brings you the most Elegant Looking Pot w...,5725,950.000000
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,[HIGH QUALITY PVC MATERIAL: The kitchen alumin...,Aluminum Foil Stickers-good kitchen helper for...,6030,984.251967
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,[100% Pure And Natural Essential Oil Or Fragra...,"Transform your home, workplace or hotel room i...",8201,393.700787


In [70]:
## Make A single sentence from the description, bullet points and title

df_train['TAGS'] = df_train['TITLE'] + ' ' + df_train['BULLET_POINTS'] + ' ' + df_train['DESCRIPTION']

In [71]:
df_train.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,TAGS
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495,PRIKNIK Horn Red Electric Air Horn Compressor ...
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574,ALISHAH Women's Cotton Ankle Length Leggings C...
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,"[Simple and elegant, great for displaying indo...",HINS Brings you the most Elegant Looking Pot w...,5725,950.000000,HINS Metal Bucket Shape Plant Pot for Indoor &...
7,2026580,Delavala Self Adhesive Kitchen Backsplash Wall...,[HIGH QUALITY PVC MATERIAL: The kitchen alumin...,Aluminum Foil Stickers-good kitchen helper for...,6030,984.251967,Delavala Self Adhesive Kitchen Backsplash Wall...
9,2998633,Hexwell Essential oil for Home Fragrance Oil A...,[100% Pure And Natural Essential Oil Or Fragra...,"Transform your home, workplace or hotel room i...",8201,393.700787,Hexwell Essential oil for Home Fragrance Oil A...


In [72]:
df_train.drop(['TITLE', 'BULLET_POINTS', 'DESCRIPTION'], axis=1, inplace=True)

In [73]:
df_train.head()

,PRODUCT_ID,PRODUCT_TYPE_ID,PRODUCT_LENGTH,TAGS
2,2765088,7537,748.031495,PRIKNIK Horn Red Electric Air Horn Compressor ...
3,1594019,2996,787.401574,ALISHAH Women's Cotton Ankle Length Leggings C...
5,2152929,5725,950.000000,HINS Metal Bucket Shape Plant Pot for Indoor &...
7,2026580,6030,984.251967,Delavala Self Adhesive Kitchen Backsplash Wall...
9,2998633,8201,393.700787,Hexwell Essential oil for Home Fragrance Oil A...


In [74]:
df_train['TAGS'] = df_train['TAGS'].str.replace("\r", " ")
df_train['TAGS'] = df_train['TAGS'].str.replace("\n", " ")
df_train['TAGS'] = df_train['TAGS'].str.replace("    ", " ")
df_train['TAGS'] = df_train['TAGS'].str.replace('"', '')
df_train['TAGS'] = df_train['TAGS'].str.lower()

In [75]:
df_train.head()


,PRODUCT_ID,PRODUCT_TYPE_ID,PRODUCT_LENGTH,TAGS
2,2765088,7537,748.031495,priknik horn red electric air horn compressor ...
3,1594019,2996,787.401574,alishah women's cotton ankle length leggings c...
5,2152929,5725,950.000000,hins metal bucket shape plant pot for indoor &...
7,2026580,6030,984.251967,delavala self adhesive kitchen backsplash wall...
9,2998633,8201,393.700787,hexwell essential oil for home fragrance oil a...


In [76]:
df_train.drop(['PRODUCT_TYPE_ID'], axis=1, inplace=True)

In [77]:
df_train.head()

,PRODUCT_ID,PRODUCT_LENGTH,TAGS
2,2765088,748.031495,priknik horn red electric air horn compressor ...
3,1594019,787.401574,alishah women's cotton ankle length leggings c...
5,2152929,950.000000,hins metal bucket shape plant pot for indoor &...
7,2026580,984.251967,delavala self adhesive kitchen backsplash wall...
9,2998633,393.700787,hexwell essential oil for home fragrance oil a...


In [78]:
df_test = pd.read_csv('test.csv')

In [79]:
df_test.head()


,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN,6142
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,1622
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN,6318


In [80]:
df_test.isnull().sum()

PRODUCT_ID              0
TITLE                   5
BULLET_POINTS      275922
DESCRIPTION        380001
PRODUCT_TYPE_ID         0
dtype: int64

In [81]:
df_test['DESCRIPTION'] = df_test['DESCRIPTION'].str.replace(r'<[^<>]*>', '', regex=True)

In [82]:
df_test

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN,6142
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,DCGARING Throw Blanket Size Chart ( W x L ): T...,1622
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN,6318
...,...,...,...,...,...
734731,921419,"Casual Canine Basic Hoodie for Dogs, 16"" Mediu...",[Brightly colored pet sweatshirts with authent...,"ZA6015 16 43 Size-See Chart Below: Medium: 16""...",7073
734732,2456362,Dive Log Book: Scuba Diving Logbook for Beginn...,NaN,NaN,1
734733,841529,Axor 39135001 Citterio Widespread Faucet with ...,"[8-Inch centers,1/2-Inch IPS inlets,9-Inch spo...",39135001 Features: -ADA compliant. -Includes p...,10645
734734,1190194,Caroline's Treasures BB1801DS812 Halloween Bas...,"[Indoor or outdoor aluminum artwork prints,8 i...",Features. Great for inside or outside these Al...,12680


In [83]:
df_test['TAGS'] = df_test['TITLE'] + ' ' + df_test['BULLET_POINTS'] + ' ' + df_test['DESCRIPTION']

In [84]:
df_test.drop(['TITLE', 'BULLET_POINTS', 'DESCRIPTION'], axis=1, inplace=True)

In [85]:
df_test.head()

,PRODUCT_ID,PRODUCT_TYPE_ID,TAGS
0,604373,6142,NaN
1,1729783,1622,DCGARING Microfiber Throw Blanket Warm Fuzzy P...
2,1871949,7540,I-Match Auto Parts Front License Plate Bracket...
3,1107571,12442,PinMart Gold Plated Excellence in Service 1 Ye...
4,624253,6318,NaN


In [86]:
df_test.drop(['PRODUCT_TYPE_ID'], axis=1, inplace=True)


In [87]:
df_test.head()

,PRODUCT_ID,TAGS
0,604373,NaN
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...
2,1871949,I-Match Auto Parts Front License Plate Bracket...
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...
4,624253,NaN


In [88]:
df_test['TAGS'] = df_test['TAGS'].str.replace("\r", " ")
df_test['TAGS'] = df_test['TAGS'].str.replace("\n", " ")
df_test['TAGS'] = df_test['TAGS'].str.replace("    ", " ")
df_test['TAGS'] = df_test['TAGS'].str.replace('"', '')
df_test['TAGS'] = df_test['TAGS'].str.lower()

In [89]:
df_test.head()

,PRODUCT_ID,TAGS
0,604373,NaN
1,1729783,dcgaring microfiber throw blanket warm fuzzy p...
2,1871949,i-match auto parts front license plate bracket...
3,1107571,pinmart gold plated excellence in service 1 ye...
4,624253,NaN


In [90]:
df_test.isna().sum()

PRODUCT_ID         0
TAGS          397734
dtype: int64

In [91]:
df_test


,PRODUCT_ID,TAGS
0,604373,NaN
1,1729783,dcgaring microfiber throw blanket warm fuzzy p...
2,1871949,i-match auto parts front license plate bracket...
3,1107571,pinmart gold plated excellence in service 1 ye...
4,624253,NaN
...,...,...
734731,921419,"casual canine basic hoodie for dogs, 16 medium..."
734732,2456362,NaN
734733,841529,axor 39135001 citterio widespread faucet with ...
734734,1190194,caroline's treasures bb1801ds812 halloween bas...


In [92]:
df_train

,PRODUCT_ID,PRODUCT_LENGTH,TAGS
2,2765088,748.031495,priknik horn red electric air horn compressor ...
3,1594019,787.401574,alishah women's cotton ankle length leggings c...
5,2152929,950.000000,hins metal bucket shape plant pot for indoor &...
7,2026580,984.251967,delavala self adhesive kitchen backsplash wall...
9,2998633,393.700787,hexwell essential oil for home fragrance oil a...
...,...,...,...
2249686,1898841,600.000000,cocomii holographic iphone 11 pro case - holog...
2249689,1616561,1200.000000,have it tall men's curved hem long drop tail t...
2249694,2766635,125.984252,"(3pcs) goose game cute cartoon enamel pins, fu..."
2249695,1987786,1200.000000,kangroo sweep movement printed wooden wall clo...


In [93]:
df_train.isna().sum()

PRODUCT_ID        0
PRODUCT_LENGTH    0
TAGS              0
dtype: int64

In [95]:
df_train['TAGS']=df_train['TAGS'].str.replace("\r", " ")


In [96]:
df_test['TAGS'].str.replace("\r", " ")

0                                                       NaN
1         dcgaring microfiber throw blanket warm fuzzy p...
2         i-match auto parts front license plate bracket...
3         pinmart gold plated excellence in service 1 ye...
4                                                       NaN
                                ...                        
734731    casual canine basic hoodie for dogs, 16 medium...
734732                                                  NaN
734733    axor 39135001 citterio widespread faucet with ...
734734    caroline's treasures bb1801ds812 halloween bas...
734735    amsahr 18c1623 lexmark x3530 remanufactured re...
Name: TAGS, Length: 734736, dtype: object

In [101]:
df_test.drop(['Temp'], axis=1, inplace=True)

In [102]:
df_test

,PRODUCT_ID,TAGS
0,604373,NaN
1,1729783,dcgaring microfiber throw blanket warm fuzzy p...
2,1871949,i-match auto parts front license plate bracket...
3,1107571,pinmart gold plated excellence in service 1 ye...
4,624253,NaN
...,...,...
734731,921419,"casual canine basic hoodie for dogs, 16 medium..."
734732,2456362,NaN
734733,841529,axor 39135001 citterio widespread faucet with ...
734734,1190194,caroline's treasures bb1801ds812 halloween bas...


In [106]:
df_test.drop(['Temp'], axis=1, inplace=True)

In [107]:
df_test

,PRODUCT_ID,TAGS
0,604373,NaN
1,1729783,dcgaring microfiber throw blanket warm fuzzy p...
2,1871949,i-match auto parts front license plate bracket...
3,1107571,pinmart gold plated excellence in service 1 ye...
4,624253,NaN
...,...,...
734731,921419,"casual canine basic hoodie for dogs, 16 medium..."
734732,2456362,NaN
734733,841529,axor 39135001 citterio widespread faucet with ...
734734,1190194,caroline's treasures bb1801ds812 halloween bas...


In [108]:
df_test['T'] = df_test['TAGS'] + ' ' + df_test['PRODUCT_ID'].astype(str)